In [3]:
import pandas as pd
import glob
import os

# --- Konfigurasi ---
path_to_files = './' 
file_pattern = '/*.csv' 
output_filename = 'data_gabungan_imputasi_city_fixed.csv' # Nama file output baru

# Daftar untuk menampung semua DataFrame
all_data = []

# Cari semua file CSV di direktori
full_pattern = path_to_files + file_pattern
file_list = glob.glob(full_pattern)

print(f"Ditemukan {len(file_list)} file CSV untuk digabungkan.")

# Iterasi melalui setiap file (Sama seperti sebelumnya)
for filename in file_list:
    try:
        base_name = os.path.basename(filename)
        city = os.path.splitext(base_name)[0]
        city_clean = city.replace('_', ' ').title()
        df = pd.read_csv(filename) 
        df['City'] = city_clean
        all_data.append(df)
        print(f"  Berhasil memproses: {base_name}. City: {city_clean}")

    except pd.errors.EmptyDataError:
        print(f"  Peringatan: File {filename} kosong. Melewati.")
    except Exception as e:
        print(f"  Gagal memproses file {filename}: {e}")

# --- Proses Penggabungan Utama ---
if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)

    # --- Langkah Perbaikan/Imputasi ---
    print("-" * 50)
    print("Memproses imputasi (mengisi data kosong) untuk ISPU PM2.5 dan AOD...")

    kolom_imputasi = ['ISPU PM2.5', 'AOD']
    
    for kolom in kolom_imputasi:
        if kolom in df_combined.columns:
            # 1. Konversi ke numerik
            df_combined[kolom] = pd.to_numeric(df_combined[kolom], errors='coerce')
            
            # 2. Hitung rata-rata
            average_value = df_combined[kolom].mean()
            
            if pd.notna(average_value):
                # 3. PERBAIKAN: Mengisi nilai NaN dan menugaskannya kembali ke kolom
                # Ini adalah cara yang direkomendasikan dan menghilangkan FutureWarning
                df_combined[kolom] = df_combined[kolom].fillna(average_value) 
                
                print(f"  ✅ Kolom '{kolom}' diisi dengan rata-rata: {average_value:.4f}")
            else:
                print(f"  ❌ Kolom '{kolom}' tidak dapat dihitung rata-ratanya.")
        else:
            print(f"  ⚠️ Peringatan: Kolom '{kolom}' tidak ditemukan di data gabungan.")


    # 5. Simpan DataFrame hasil gabungan
    df_combined.to_csv(output_filename, index=False)

    print("-" * 50)
    print(f"✅ Penggabungan & Imputasi Selesai. Data disimpan ke: {output_filename}")
    print(f"Total baris data yang digabungkan: {len(df_combined)}")
    print("-" * 50)
else:
    print("-" * 50)
    print("❌ Tidak ada data yang berhasil diproses atau digabungkan.")
    print("-" * 50)

Ditemukan 7 file CSV untuk digabungkan.
  Berhasil memproses: bundaran_hi.csv. City: Bundaran Hi
  Berhasil memproses: data_gabungan_city.csv. City: Data Gabungan City
  Berhasil memproses: data_gabungan_imputasi_city.csv. City: Data Gabungan Imputasi City
  Berhasil memproses: jagakarsa.csv. City: Jagakarsa
  Berhasil memproses: kebun_jeruk.csv. City: Kebun Jeruk
  Berhasil memproses: kelapa_gading.csv. City: Kelapa Gading
  Berhasil memproses: lubang_buaya.csv. City: Lubang Buaya
--------------------------------------------------
Memproses imputasi (mengisi data kosong) untuk ISPU PM2.5 dan AOD...
  ✅ Kolom 'ISPU PM2.5' diisi dengan rata-rata: 70.7749
  ✅ Kolom 'AOD' diisi dengan rata-rata: 0.5530
--------------------------------------------------
✅ Penggabungan & Imputasi Selesai. Data disimpan ke: data_gabungan_imputasi_city_fixed.csv
Total baris data yang digabungkan: 21920
--------------------------------------------------
